In [36]:
import glob
import os

import numpy as np                  # Numerical operations
import pandas as pd                 # Data manipulation

import matplotlib.pyplot as plt     # Basic plotting
import seaborn as sns               # Statistical data visualization

from scipy import stats             # Statistical functions
from scipy.stats import norm, ttest_ind, chi2_contingency  # Specific tests

import statsmodels.api as sm        # Statistical models
import statsmodels.formula.api as smf  # Formula-based models
from statsmodels.tsa.seasonal import seasonal_decompose  # Seasonal decomposition
from statsmodels.tsa.stattools import adfuller, kpss      # Stationarity tests
from statsmodels.tsa.arima.model import ARIMA             # ARIMA models
from statsmodels.tsa.holtwinters import ExponentialSmoothing  # Exponential smoothing

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score  # Data splitting and tuning
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder          # Data preprocessing
from sklearn.pipeline import Pipeline                                                # ML pipelines
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score   # Classification metrics
from sklearn.metrics import mean_squared_error, r2_score                              # Regression metrics
from sklearn.linear_model import LinearRegression, LogisticRegression                 # Linear models
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor                # Decision trees
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier       # Ensemble methods
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor         # Ensemble methods
from sklearn.svm import SVC, SVR                                                     # Support Vector Machines
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor               # K-Nearest Neighbors
from sklearn.cluster import KMeans, DBSCAN                                           # Clustering
from sklearn.decomposition import PCA, TruncatedSVD                                  # Dimensionality reduction

from sklearn.metrics import confusion_matrix, classification_report         # Model evaluation
from sklearn.metrics import roc_auc_score, roc_curve, auc                   # AUC and ROC
from sklearn.metrics import mean_absolute_error, mean_squared_error         # Regression evaluation


## GDP data

In [89]:
gdp_data = pd.read_excel('./data/gdp/gdp.xlsx')
gdp_data.iloc[:, 1:] = gdp_data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
deflator = gdp_data.iloc[-1, 1:]
gdp_data.iloc[:-1, 1:] = gdp_data.iloc[:-1, 1:].div(deflator, axis=1) * 100 * 1000
gdp_data = gdp_data[:-1]
gdp_data

,Регион,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Абай,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1611977787.532923,1991439766.081871,2508432318.710833
1,Акмолинская,489937625.41806,676833052.985702,763327576.335878,872712511.415525,993438374.291115,1100122669.283611,1183393133.802817,1432383579.335793,1556670238.095238,1797007620.817844,2191880806.142035,2351293327.480246,2911088137.00919,3456063025.962399
2,Актюбинская,981264966.555184,1314009167.367535,1676259637.40458,1683947671.232877,1820642344.045369,1736187634.936212,1823165316.901409,2160414391.143911,2480270512.820512,2764331691.449814,2837689251.439539,3148571202.809482,3689974436.090226,3808535989.256938
3,Алматинская,834207357.859532,1040069638.351556,1381412881.679389,1590804109.589041,1805639130.434783,1939202845.92738,1927821390.845071,2280481365.313653,2559630952.380952,3016803345.724907,3580652111.324376,2967282089.552238,3565301002.506266,4672596866.606982
4,Атырауская,2377633110.367893,3188867619.848612,3447911545.801527,3632287305.936073,4102668241.965974,4138148675.171737,4578057394.366198,5486765498.154981,7160084340.65934,8668460315.98513,7426352399.232246,9330626338.893766,11466499415.204678,12826566069.829901
5,Западно-Казахстанская,876905936.45485,1142462994.1127,1681967080.152672,1626074611.872146,1878738846.880907,1678069381.746811,1789322095.070422,2156370571.95572,2555550915.750916,2738279832.713755,2625674760.076775,3101856365.23266,3705205179.615706,4765617278.424351
6,Жамбылская,373243561.87291,532879394.449117,725569942.748092,800672420.091324,925960396.975425,995588420.019627,1041196214.788732,1245997785.97786,1403039010.989011,1591899256.505576,1824745681.381958,1986611589.113257,2243491729.323308,2731968576.544315
7,Жетісу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1077312115.891133,1192048621.553885,1528555416.293644
8,Карагандинская,1565921655.518395,2008162489.486964,2334456393.129771,2394418995.43379,2740998865.784499,3049151717.36997,3267654841.549296,3952364022.140221,4335532967.032968,5007677137.546469,5853988675.6238,5328671290.605794,6080249958.228906,7277354431.51298
9,Костанайская,716343979.93311,953973591.253154,1097976431.29771,1231315068.493151,1318400567.107751,1352559764.474975,1340037059.859155,1706901291.512915,1894950732.600733,2278565427.509294,2756439155.47025,3087112379.28007,3493799331.662489,4173525962.399284


In [90]:
gdp_data = gdp_data.melt(id_vars=['Регион'], var_name='Год', value_name='ВВП')

In [91]:
gdp_data

,Регион,Год,ВВП
0,Абай,2010,NaN
1,Акмолинская,2010,489937625.41806
2,Актюбинская,2010,981264966.555184
3,Алматинская,2010,834207357.859532
4,Атырауская,2010,2377633110.367893
...,...,...,...
289,Ұлытау,2023,1857578871.978514
290,Восточно-Казахстанская,2023,4150631692.032228
291,г. Астана,2023,11567001790.510296
292,г. Алматы,2023,22288262846.911369


In [92]:
gdp_data['ВВП_прош'] = gdp_data.groupby("Регион")["ВВП"].shift(1)
gdp_data

,Регион,Год,ВВП,ВВП_прош
0,Абай,2010,NaN,NaN
1,Акмолинская,2010,489937625.41806,NaN
2,Актюбинская,2010,981264966.555184,NaN
3,Алматинская,2010,834207357.859532,NaN
4,Атырауская,2010,2377633110.367893,NaN
...,...,...,...,...
289,Ұлытау,2023,1857578871.978514,1344811862.99081
290,Восточно-Казахстанская,2023,4150631692.032228,3272195572.263993
291,г. Астана,2023,11567001790.510296,8916023809.523809
292,г. Алматы,2023,22288262846.911369,16002119214.703426


## Population data

In [93]:
population_data = pd.read_excel('./data/population/population.xlsx')
population_data.iloc[:, 1:] = population_data.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
population_data

,Регион,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Абай,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,611888,610198,607589
1,Акмолинская,735134.0,733281.0,731399.0,732768.0,735612.0,736560.0,744420.0,734369.0,738942.0,738587.0,736735.0,735566.0,785708,788013,787976
2,Актюбинская,763591.0,777501.0,786390.0,795871.0,808985.0,822557.0,834808.0,845679.0,857711.0,869637.0,881651.0,894333.0,916750,928159,939405
3,Алматинская,1836162.0,1872844.0,1908785.0,1946831.0,1984572.0,1921888.0,1947552.0,1983465.0,2017278.0,2038935.0,2055724.0,2077967.0,1478496,1505896,1531167
4,Атырауская,520988.0,532033.0,542959.0,555199.0,567831.0,581389.0,594511.0,607528.0,620684.0,633791.0,645280.0,657110.0,681241,693079,704074
5,Западно-Казахстанская,603858.0,608334.0,612581.0,617735.0,624085.0,630056.0,636980.0,641513.0,646927.0,652325.0,656844.0,661316.0,683327,688127,693261
6,Жамбылская,1034487.0,1046260.0,1055822.0,1069785.0,1084369.0,1098489.0,1110749.0,1115307.0,1117218.0,1125440.0,1130099.0,1139192.0,1209665,1218158,1222593
7,Жетісу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,698757,698726,697987
8,Карагандинская,1346822.0,1352217.0,1357969.0,1362777.0,1369658.0,1378121.0,1384810.0,1382734.0,1380537.0,1378532.0,1376882.0,1375938.0,1134966,1134855,1135351
9,Костанайская,883379.0,881656.0,879635.0,880116.0,880968.0,881569.0,883806.0,879134.0,875616.0,872795.0,868549.0,864550.0,835686,832234,829984


In [94]:
population_data = population_data.melt(id_vars=['Регион'], var_name='Год', value_name='Население')

In [95]:
population_data

,Регион,Год,Население
0,Абай,2010,NaN
1,Акмолинская,2010,735134.0
2,Актюбинская,2010,763591.0
3,Алматинская,2010,1836162.0
4,Атырауская,2010,520988.0
...,...,...,...
295,Ұлытау,2024,221582
296,Восточно-Казахстанская,2024,727053
297,г. Астана,2024,1430117
298,г. Алматы,2024,2228677


## Investment data

In [109]:
investment_files = glob.glob(os.path.join('./data/investment/', "*.xlsx"))

# Initialize an empty list to store each year's data
all_data = []

# Loop over each file and load it
for file in investment_files:
    # Extract the year from the filename (assuming it’s in the filename, e.g., 'data_2020.xlsx')
    year = file.split('\\')[-1].replace('.xlsx', '')
    
    # Load the Excel file into a DataFrame
    df = pd.read_excel(file)
    
    # Add a 'Year' column
    df['Год'] = int(year)
    
    # Melt the DataFrame to make it long-format: one row per sector, region, and year
    df = df.melt(id_vars=['Год', 'Отрасль'], var_name='Регион', value_name='Инвестиции')
    
    # Append the processed DataFrame to the list
    all_data.append(df)

# Concatenate all years' data into a single DataFrame
investment_data = pd.concat(all_data, ignore_index=True)
investment_data["Отрасль"] = investment_data["Отрасль"].replace({
    "Водоснабжение; водоотведение; сбор, обработка и удаление отходов, деятельность по ликвидации загрязнений": "Водоснабжение; сбор, обработка и удаление отходов, деятельность по ликвидации загрязнений",
})
investment_data['Инвестиции'] = investment_data['Инвестиции'].apply(pd.to_numeric, errors='coerce')
investment_data

,Год,Отрасль,Регион,Инвестиции
0,2009,"Сельское, лесное и рыбное хозяйство",Акмолинская,19959652.0
1,2009,Горнодобывающая промышленность и разработка ка...,Акмолинская,25886719.0
2,2009,Обрабатывающая промышленность,Акмолинская,16269252.0
3,2009,"Снабжение электроэнергией, газом, паром, горяч...",Акмолинская,3876685.0
4,2009,"Водоснабжение; сбор, обработка и удаление отхо...",Акмолинская,15846029.0
...,...,...,...,...
4783,2023,Государственное управление и оборона; обязател...,г. Шымкент,2764003.0
4784,2023,Образование,г. Шымкент,25582355.0
4785,2023,Здравоохранение и социальное обслуживание насе...,г. Шымкент,11564393.0
4786,2023,"Искусство, развлечения и отдых",г. Шымкент,18236633.0


In [111]:
investment_data = investment_data.pivot_table(index=['Год', 'Регион'], columns='Отрасль', values='Инвестиции', fill_value=0)
investment_data.reset_index(inplace=True)
investment_data

Отрасль,Год,Регион,"Водоснабжение; сбор, обработка и удаление отходов, деятельность по ликвидации загрязнений",Горнодобывающая промышленность и разработка карьеров,Государственное управление и оборона; обязательное социальное обеспечение,Деятельность в области административного и вспомогательного обслуживания,Здравоохранение и социальное обслуживание населения,Информация и связь,"Искусство, развлечения и отдых",Обрабатывающая промышленность,...,Операции с недвижимым имуществом,Оптовая и розничная торговля; ремонт автомобилей и мотоциклов,Предоставление прочих видов услуг,Предоставление услуг по проживанию и питанию,"Профессиональная, научная и техническая деятельность","Сельское, лесное и рыбное хозяйство","Снабжение электроэнергией, газом, паром, горячей водой и кондиционированным воздухом",Строительство,Транспорт и складирование,Финансовая и страховая деятельность
0,2009,Акмолинская,15846029,25886719.0,1689597,140374,1552799,1150417,2864947,16269252.0,...,8876807,2128348,108647,432176,122046,19959652,3876685,715387,43342070,806197
1,2009,Актюбинская,10331654,169509992.0,1108487,123533,5938693,1551947,2957863,7283494.0,...,15158490,7672373,555835,95431,21831853,2688106,12510618,2499861,42907086,850753
2,2009,Алматинская,8206769,685040.0,3814392,1184621,5921525,2041042,8571721,22866059.0,...,48809611,14643819,523483,3457178,190094,5410948,12777279,5384733,159140444,686282
3,2009,Атырауская,8235014,732300902.0,3157137,395519,4368936,2004761,2995590,22410949.0,...,17940277,1816549,170655,1003747,153684304,409656,21120602,6771858,111886967,361595
4,2009,Восточно-Казахстанская,3496364,18307006.0,1740776,153146,6866508,2265510,2056125,61933595.0,...,6989927,3488638,768731,1147634,3085424,2648296,6863580,1945166,7835733,1387547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2023,Туркестанская,69419896,145089830.0,22061800,28270157,24590647,3885578,27657258,90407013.0,...,155318434,36833238,19201028,24877577,4504725,93972629,65637244,1690269,66172787,380594
248,2023,г. Алматы,36402483,0.0,23868527,24968306,57400086,320558211,23321446,60151240.0,...,687628822,114894907,5915302,25057862,23401776,1457,57217373,17175055,198306412,70405167
249,2023,г. Астана,69393564,0.0,15405875,23951455,141719687,47297468,82411745,39724853.0,...,516830228,29724106,168776,3673994,12300533,0,148067649,6184330,456888179,26110423
250,2023,г. Шымкент,31921802,1107622.0,2764003,3190821,11564393,12350223,18236633,64568111.0,...,298389009,30036783,36257030,4876576,1171672,6099155,18024122,5715446,88476674,1407839


In [97]:
print(investment_data['Отрасль'].unique())
print(len(investment_data['Отрасль'].unique()))

['Сельское, лесное и рыбное хозяйство'
 'Горнодобывающая промышленность и разработка карьеров'
 'Обрабатывающая промышленность'
 'Снабжение электроэнергией, газом, паром, горячей водой и кондиционированным воздухом'
 'Водоснабжение; сбор, обработка и удаление отходов, деятельность по ликвидации загрязнений'
 'Строительство'
 'Оптовая и розничная торговля; ремонт автомобилей и мотоциклов'
 'Транспорт и складирование'
 'Предоставление услуг по проживанию и питанию' 'Информация и связь'
 'Финансовая и страховая деятельность' 'Операции с недвижимым имуществом'
 'Профессиональная, научная и техническая деятельность'
 'Деятельность в области административного и вспомогательного обслуживания'
 'Государственное управление и оборона; обязательное  социальное обеспечение'
 'Образование' 'Здравоохранение и социальное обслуживание населения'
 'Искусство, развлечения и отдых' 'Предоставление прочих видов услуг']
19


## Merging

In [112]:
# temp = pd.merge(investment_data, population_data, on=['Год', 'Регион'], how='inner')
merged = pd.merge(investment_data, gdp_data, on=['Год', 'Регион'], how='inner')
merged

,Год,Регион,"Водоснабжение; сбор, обработка и удаление отходов, деятельность по ликвидации загрязнений",Горнодобывающая промышленность и разработка карьеров,Государственное управление и оборона; обязательное социальное обеспечение,Деятельность в области административного и вспомогательного обслуживания,Здравоохранение и социальное обслуживание населения,Информация и связь,"Искусство, развлечения и отдых",Обрабатывающая промышленность,...,Предоставление прочих видов услуг,Предоставление услуг по проживанию и питанию,"Профессиональная, научная и техническая деятельность","Сельское, лесное и рыбное хозяйство","Снабжение электроэнергией, газом, паром, горячей водой и кондиционированным воздухом",Строительство,Транспорт и складирование,Финансовая и страховая деятельность,ВВП,ВВП_прош
0,2010,Акмолинская,14212757,5016032.0,1242809,1311978,2908873,933114,1415573,15169234.0,...,80816,1867435,811633,18348378,4714132,954630,24494800,503836,489937625.41806,NaN
1,2010,Алматинская,9543110,383705.0,1425433,3389655,16674035,2780961,16354781,29226333.0,...,154502,1598125,2359167,6901042,38655064,5064048,80136148,572239,834207357.859532,NaN
2,2010,Атырауская,7759441,837182300.0,2338927,930581,6138562,1696915,1296481,27214088.0,...,123716,705937,35504872,374334,27277212,6670426,117299193,309922,2377633110.367893,NaN
3,2010,Восточно-Казахстанская,4456091,26020561.0,1970385,1324098,5089105,3301856,1841858,44003782.0,...,372859,715553,2848003,4722455,9223480,3035232,12258589,816957,1040219565.217391,NaN
4,2010,Жамбылская,3968089,1630266.0,2116330,106381,6184176,1996009,210472,33482519.0,...,169600,210736,249646,166203,5368094,643650,80147424,195075,373243561.87291,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,2023,Туркестанская,69419896,145089830.0,22061800,28270157,24590647,3885578,27657258,90407013.0,...,19201028,24877577,4504725,93972629,65637244,1690269,66172787,380594,3629331154.87914,2938413617.376775
218,2023,г. Алматы,36402483,0.0,23868527,24968306,57400086,320558211,23321446,60151240.0,...,5915302,25057862,23401776,1457,57217373,17175055,198306412,70405167,22288262846.911369,16002119214.703426
219,2023,г. Астана,69393564,0.0,15405875,23951455,141719687,47297468,82411745,39724853.0,...,168776,3673994,12300533,0,148067649,6184330,456888179,26110423,11567001790.510296,8916023809.523809
220,2023,г. Шымкент,31921802,1107622.0,2764003,3190821,11564393,12350223,18236633,64568111.0,...,36257030,4876576,1171672,6099155,18024122,5715446,88476674,1407839,3604836078.782453,2752207435.254803


## GDP Prediction

In [113]:
data = pd.get_dummies(merged, columns=['Регион'], drop_first=True)

In [114]:
X = data.drop(columns=['ВВП'])  # Drop the target and any redundant columns
y = data['ВВП']  # Target variable

In [115]:
from sklearn.model_selection import StratifiedShuffleSplit

# Assuming 'Регион' columns were one-hot encoded, you can use the original 'Регион' column for stratification
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Since StratifiedShuffleSplit only provides indices, let's use it to split X and y
for train_index, test_index in stratified_split.split(X, merged['Регион']):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [116]:
X_train

,Год,"Водоснабжение; сбор, обработка и удаление отходов, деятельность по ликвидации загрязнений",Горнодобывающая промышленность и разработка карьеров,Государственное управление и оборона; обязательное социальное обеспечение,Деятельность в области административного и вспомогательного обслуживания,Здравоохранение и социальное обслуживание населения,Информация и связь,"Искусство, развлечения и отдых",Обрабатывающая промышленность,Образование,...,Регион_Кызылординская,Регион_Мангистауская,Регион_Павлодарская,Регион_Северо-Казахстанская,Регион_Туркестанская,Регион_Южно-Казахстанская,Регион_г. Алматы,Регион_г. Астана,Регион_г. Шымкент,Регион_Ұлытау
90,2016,10658990,11546157.0,297818,242046,2323947,1317540,6445024,25974796.0,2672237,...,False,False,False,False,False,False,False,False,False,False
139,2019,34956442,252971293.0,1039326,3870533,9938464,3811328,22445503,65747835.0,4885920,...,False,False,False,False,False,False,False,False,False,False
207,2023,26370804,104273296.0,4907704,2170177,21593813,9068834,4338617,96713719.0,5131334,...,False,False,False,False,False,False,False,False,False,False
144,2019,19438857,119700284.0,1730041,755115,8056449,1704164,2106393,18030232.0,4511151,...,True,False,False,False,False,False,False,False,False,False
34,2012,9602007,2474107.0,2014938,288277,9323121,2709027,2810330,21909573.0,10662617,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2017,15192670,2202881.0,1218380,13782197,2474080,1138332,3063443,21313742.0,3728740,...,False,False,False,True,False,False,False,False,False,False
78,2015,7242661,205876517.0,274437,2767058,3621910,9501241,1269974,43505865.0,13001172,...,False,False,False,False,False,False,False,False,False,False
136,2019,22648912,21906895.0,376212,1831241,7513432,2274818,3874615,39463449.0,7786743,...,False,False,False,False,False,False,False,False,False,False
189,2022,29324641,31437569.0,3367655,781737,20738144,1530796,8082273,120044109.0,13495809,...,False,False,False,False,False,False,False,False,False,False


In [127]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Initialize the model
model = RandomForestRegressor(random_state=42, n_estimators=80)

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mse}")

Mean Absolute Error: 474061432.9779968
